In [ ]:
! kaggle datasets download alincijov/bilingual-sentence-pairs

Dataset URL: https://www.kaggle.com/datasets/alincijov/bilingual-sentence-pairs
License(s): unknown
 92% 92.0M/99.7M [00:01<00:00, 79.4MB/s]
100% 99.7M/99.7M [00:01<00:00, 69.9MB/s]


In [ ]:
!unzip /content/bilingual-sentence-pairs.zip

Archive:  /content/bilingual-sentence-pairs.zip
  inflating: afr.txt                 
  inflating: ara.txt                 
  inflating: arq.txt                 
  inflating: asm.txt                 
  inflating: aze.txt                 
  inflating: bel.txt                 
  inflating: ben.txt                 
  inflating: ber.txt                 
  inflating: bos.txt                 
  inflating: bul.txt                 
  inflating: cat.txt                 
  inflating: cbk.txt                 
  inflating: ceb.txt                 
  inflating: ces.txt                 
  inflating: chv.txt                 
  inflating: cmn.txt                 
  inflating: dan.txt                 
  inflating: deu.txt                 
  inflating: dtp.txt                 
  inflating: ell.txt                 
  inflating: est.txt                 
  inflating: eus.txt                 
  inflating: fin.txt                 
  inflating: fra.txt                 
  inflating: glg.txt                 
  

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import __future__
print(__future__.all_feature_names)

['nested_scopes', 'generators', 'division', 'absolute_import', 'with_statement', 'print_function', 'unicode_literals', 'barry_as_FLUFL', 'generator_stop', 'annotations']


In [ ]:
# filename='/content/ara.txt'
# file = open(filename, mode='rt', encoding='utf-8')
# # read all text
# data = file.read()
# file.close()

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
#unicode - ascii
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    print("The sentence here is: ", s)
    s = unicodeToAscii(s.lower().strip())
    # s = re.sub(r"([.!?])", r" \1", s)
    # s = re.sub(r"[^a-zA-Z!?]+", r" ", s) #trimming punctuations
    return s.strip()

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    # The filename was hardcoded to '/content/ara.txt' and did not utilize the lang1 and lang2 arguments
    # This has been changed to reflect the intended usage. You may need to modify this based on your actual file naming convention
    # lines = open('/content/ara.txt' % (lang1, lang2), encoding='utf-8').\
    lines = open('/content/ara.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines[:1000]]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
# input_lang, output_lang, pairs = prepareData('eng', 'ara', True)

input_lang, output_lang, pairs = readLangs('eng', 'ara', True)

Reading lines...
The sentence here is:  Hi.
The sentence here is:  مرحبًا.
The sentence here is:  Run!
The sentence here is:  اركض!
The sentence here is:  Help!
The sentence here is:  النجدة!
The sentence here is:  Jump!
The sentence here is:  اقفز!
The sentence here is:  Stop!
The sentence here is:  قف!
The sentence here is:  Go on.
The sentence here is:  داوم.
The sentence here is:  Go on.
The sentence here is:  استمر.
The sentence here is:  Hello!
The sentence here is:  مرحباً.
The sentence here is:  Hurry!
The sentence here is:  تعجّل!
The sentence here is:  Hurry!
The sentence here is:  استعجل!
The sentence here is:  I see.
The sentence here is:  انا اري
The sentence here is:  I won!
The sentence here is:  أنا فُزت!
The sentence here is:  Relax.
The sentence here is:  استرح.
The sentence here is:  Smile.
The sentence here is:  ابتسم.
The sentence here is:  Cheers!
The sentence here is:  في صحتك.
The sentence here is:  Got it?
The sentence here is:  هل فهمت؟
The sentence here is:  

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")

    for pair in pairs:
        print(pair)
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'ara', True)
print(random.choice(pairs))

Reading lines...
The sentence here is:  Hi.
The sentence here is:  مرحبًا.
The sentence here is:  Run!
The sentence here is:  اركض!
The sentence here is:  Help!
The sentence here is:  النجدة!
The sentence here is:  Jump!
The sentence here is:  اقفز!
The sentence here is:  Stop!
The sentence here is:  قف!
The sentence here is:  Go on.
The sentence here is:  داوم.
The sentence here is:  Go on.
The sentence here is:  استمر.
The sentence here is:  Hello!
The sentence here is:  مرحباً.
The sentence here is:  Hurry!
The sentence here is:  تعجّل!
The sentence here is:  Hurry!
The sentence here is:  استعجل!
The sentence here is:  I see.
The sentence here is:  انا اري
The sentence here is:  I won!
The sentence here is:  أنا فُزت!
The sentence here is:  Relax.
The sentence here is:  استرح.
The sentence here is:  Smile.
The sentence here is:  ابتسم.
The sentence here is:  Cheers!
The sentence here is:  في صحتك.
The sentence here is:  Got it?
The sentence here is:  هل فهمت؟
The sentence here is:  

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [ ]:
MAX_LENGTH = 20

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'ara', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 200, print_every=5, plot_every=5)

Reading lines...
The sentence here is:  Hi.
The sentence here is:  مرحبًا.
The sentence here is:  Run!
The sentence here is:  اركض!
The sentence here is:  Help!
The sentence here is:  النجدة!
The sentence here is:  Jump!
The sentence here is:  اقفز!
The sentence here is:  Stop!
The sentence here is:  قف!
The sentence here is:  Go on.
The sentence here is:  داوم.
The sentence here is:  Go on.
The sentence here is:  استمر.
The sentence here is:  Hello!
The sentence here is:  مرحباً.
The sentence here is:  Hurry!
The sentence here is:  تعجّل!
The sentence here is:  Hurry!
The sentence here is:  استعجل!
The sentence here is:  I see.
The sentence here is:  انا اري
The sentence here is:  I won!
The sentence here is:  أنا فُزت!
The sentence here is:  Relax.
The sentence here is:  استرح.
The sentence here is:  Smile.
The sentence here is:  ابتسم.
The sentence here is:  Cheers!
The sentence here is:  في صحتك.
The sentence here is:  Got it?
The sentence here is:  هل فهمت؟
The sentence here is:  

In [ ]:
#Set dropout layers to eval mode
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> تضع الطيور البيض.
= birds lay eggs.
< birds <EOS>

> انا سعيد للغاية.
= i'm very happy.
< i'm very happy. <EOS>

> ذاك مبلل.
= that's wet.
< that's <EOS>

> انا احبك.
= i love you.
< i love you. <EOS>

> نحن اتون
= we're coming.
< we're <EOS>

> لقد غادر توم.
= tom left.
< tom SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> يا لها من راحة
= what a relief!
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> تحقق مجددا.
= check again.
< check <EOS>

> ماذا عنك؟
= how about you?
< how SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> دعني اذهب.
= let me go.
< let go. <EOS>

